In [42]:
import os
import sys
while os.path.split(os.getcwd())[1] != 'RecSysChallenge2023-Team':
    os.chdir('..')
sys.path.insert(1, os.getcwd())

import Daniele.Utils.MatrixManipulation as mm
import Daniele.Utils.MyDataManager as dm

from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout
from sklearn.preprocessing import normalize
import similaripy
import numpy as np

URMv = dm.getURMviews()
URMo = dm.getURMopen()
#URM_all = mm.defaultExplicitURM(urmv=URMv, urmo=URMo, normalize=False, add_aug=False)

URM_all = URMv + URMo
URM_all.data = np.ones(len(URM_all.data))


ICMt=dm.getICMt()
ICMl=dm.getICMl()

URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(URM_validation, [10])
evaluator_test = EvaluatorHoldout(URM_test, [10])

EvaluatorHoldout: Ignoring 728 ( 1.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 320 ( 0.8%) Users that have less than 1 test interactions


In [43]:
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender

recommender = MultiThreadSLIM_SLIMElasticNetRecommender
#recommender.fit(alpha=0.080068, l1_ratio=0.004213)

In [44]:
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from skopt.space import Real, Integer, Categorical

metric_to_optimize = "MAP_MIN_DEN"

hyperparameterSearch = SearchBayesianSkopt(recommender,
                                           evaluator_validation=evaluator_validation,
                                           evaluator_test=evaluator_test)

earlystopping_keywargs = {"validation_every_n": 5,
                          "stop_on_validation": True,
                          "evaluator_object": evaluator_validation,
                          "lower_validations_allowed": 4,
                          "validation_metric": metric_to_optimize,
                          }

recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[URM_train],  # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={},
    EARLYSTOPPING_KEYWORD_ARGS={},
)

recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[URM_train_validation],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={},
    EARLYSTOPPING_KEYWORD_ARGS={},
)

hyperparameters_range_dictionary = {
    "alpha": Real(low=1e-0, high=1e+1, prior='uniform'),
    "l1_ratio": Real(low=1e-3, high=9e-3, prior='uniform'),
    "topK": Integer(500, 1000),
}
output_folder_path = "Federico/Recommenders/SLIM_ElasticNet/result_experiments_binary_positive_alpha/"
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

In [45]:
n_cases = 2000
n_random_starts = int(n_cases * 0.3)
cutoff_to_optimize = 10

hyperparameterSearch.search(
    recommender_input_args,
    recommender_input_args_last_test=recommender_input_args_last_test,
    hyperparameter_search_space=hyperparameters_range_dictionary,
    n_cases=n_cases,
    n_random_starts=n_random_starts,
    save_model="no",
    output_folder_path=output_folder_path,  # Where to save the results
    output_file_name_root="massive_optimization",  # How to call the files
    metric_to_optimize=metric_to_optimize,
    cutoff_to_optimize=cutoff_to_optimize,
    resume_from_saved=True
)


SearchBayesianSkopt: Resuming 'massive_optimization' Failed, no such file exists.

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'alpha': 4.682605799703209, 'l1_ratio': 0.007948492067047537, 'topK': 840}
SLIMElasticNetRecommender: URM Detected 3461 (12.4%) items with no interactions.




  0%|          | 0/27968 [00:00<?, ?it/s]

  0%|          | 8/27968 [00:01<1:47:33,  4.33it/s]

  0%|          | 104/27968 [00:02<07:36, 61.10it/s]

  1%|          | 200/27968 [00:02<04:20, 106.42it/s]

  1%|          | 296/27968 [00:03<03:14, 142.51it/s]

  1%|▏         | 392/27968 [00:03<02:40, 171.56it/s]

  2%|▏         | 488/27968 [00:03<02:20, 195.13it/s]

  2%|▏         | 584/27968 [00:04<02:07, 214.41it/s]

  2%|▏         | 680/27968 [00:04<02:01, 224.30it/s]

  3%|▎         | 776/27968 [00:04<01:55, 235.77it/s]

  3%|▎         | 872/27968 [00:05<01:51, 243.39it/s]

  3%|▎         | 968/27968 [00:05<01:51, 242.88it/s]

  4%|▍         | 1064/27968 [00:06<01:47, 250.19it/s]

  4%|▍         | 1160/27968 [00:06<01:45, 255.19it/s]

  4%|▍         | 1256/27968 [00:06<01:44, 255.71it/s]

  5%|▍         | 1352/27968 [00:07<01:41, 261.36it/s]

  5%|▌         | 1448/27968 [00:07<01:40, 264.69it/s]

  6%|▌         | 1544/27968 [00:07<01:39, 265.76it/s]

  6%|▌         | 1640/27968 [00:0

EvaluatorHoldout: Processed 40901 (100.0%) in 20.14 sec. Users per second: 2031
SearchBayesianSkopt: New best config found. Config 0: {'alpha': 4.682605799703209, 'l1_ratio': 0.007948492067047537, 'topK': 840} - results: PRECISION: 0.0001296, PRECISION_RECALL_MIN_DEN: 0.0002920, RECALL: 0.0002731, MAP: 0.0000367, MAP_MIN_DEN: 0.0000832, MRR: 0.0003668, NDCG: 0.0002035, F1: 0.0001758, HIT_RATE: 0.0012958, ARHR_ALL_HITS: 0.0003668, NOVELTY: 0.0055202, AVERAGE_POPULARITY: 0.0038960, DIVERSITY_MEAN_INTER_LIST: 0.3112402, DIVERSITY_HERFINDAHL: 0.9311233, COVERAGE_ITEM: 0.0028604, COVERAGE_ITEM_HIT: 0.0007509, ITEMS_IN_GT: 0.8713530, COVERAGE_USER: 0.9825122, COVERAGE_USER_HIT: 0.0012732, USERS_IN_GT: 0.9825122, DIVERSITY_GINI: 0.0005019, SHANNON_ENTROPY: 4.0145061, RATIO_DIVERSITY_HERFINDAHL: 0.9314316, RATIO_DIVERSITY_GINI: 0.0011351, RATIO_SHANNON_ENTROPY: 0.2975983, RATIO_AVERAGE_POPULARITY: 0.0767172, RATIO_NOVELTY: 0.4788481, 

EvaluatorHoldout: Processed 41309 (100.0%) in 20.49 sec. U



  0%|          | 0/27968 [00:00<?, ?it/s]

  0%|          | 8/27968 [00:02<2:08:39,  3.62it/s]

  0%|          | 40/27968 [00:02<21:07, 22.04it/s] 

  0%|          | 104/27968 [00:02<07:27, 62.24it/s]

  1%|          | 200/27968 [00:02<04:01, 114.98it/s]

  1%|          | 264/27968 [00:03<02:58, 155.44it/s]

  1%|          | 296/27968 [00:03<02:53, 159.95it/s]

  1%|          | 328/27968 [00:03<02:37, 174.97it/s]

  1%|▏         | 392/27968 [00:03<02:34, 178.34it/s]

  2%|▏         | 456/27968 [00:04<02:10, 211.37it/s]

  2%|▏         | 488/27968 [00:04<02:06, 216.57it/s]

  2%|▏         | 520/27968 [00:04<02:21, 193.79it/s]

  2%|▏         | 584/27968 [00:04<01:56, 235.17it/s]

  2%|▏         | 616/27968 [00:04<02:10, 210.15it/s]

  2%|▏         | 680/27968 [00:04<01:46, 256.55it/s]

  3%|▎         | 712/27968 [00:05<01:54, 238.70it/s]

  3%|▎         | 776/27968 [00:05<01:47, 253.24it/s]

  3%|▎         | 808/27968 [00:05<01:51, 244.08it/s]

  3%|▎         | 872/27968 [00:05<01:36,

EvaluatorHoldout: Processed 40901 (100.0%) in 20.69 sec. Users per second: 1977
SearchBayesianSkopt: New best config found. Config 1: {'alpha': 1.2602209346035707, 'l1_ratio': 0.0020039042947641887, 'topK': 875} - results: PRECISION: 0.0284272, PRECISION_RECALL_MIN_DEN: 0.0460644, RECALL: 0.0428265, MAP: 0.0126004, MAP_MIN_DEN: 0.0204773, MRR: 0.0988492, NDCG: 0.0442868, F1: 0.0341719, HIT_RATE: 0.2201658, ARHR_ALL_HITS: 0.1112850, NOVELTY: 0.0036616, AVERAGE_POPULARITY: 0.3352185, DIVERSITY_MEAN_INTER_LIST: 0.7074381, DIVERSITY_HERFINDAHL: 0.9707421, COVERAGE_ITEM: 0.0059354, COVERAGE_ITEM_HIT: 0.0046482, ITEMS_IN_GT: 0.8713530, COVERAGE_USER: 0.9825122, COVERAGE_USER_HIT: 0.2163155, USERS_IN_GT: 0.9825122, DIVERSITY_GINI: 0.0013697, SHANNON_ENTROPY: 5.6444004, RATIO_DIVERSITY_HERFINDAHL: 0.9710635, RATIO_DIVERSITY_GINI: 0.0030975, RATIO_SHANNON_ENTROPY: 0.4184236, RATIO_AVERAGE_POPULARITY: 6.6008760, RATIO_NOVELTY: 0.3176240, 

EvaluatorHoldout: Processed 41309 (100.0%) in 20.82 sec.

...

EvaluatorHoldout: Processed 40901 (100.0%) in 20.12 sec. Users per second: 2033
SearchBayesianSkopt: Config 2 is suboptimal. Config: {'alpha': 7.1369510544777475, 'l1_ratio': 0.0013037553581264785, 'topK': 902} - results: PRECISION: 0.0155131, PRECISION_RECALL_MIN_DEN: 0.0241641, RECALL: 0.0222026, MAP: 0.0077068, MAP_MIN_DEN: 0.0121031, MRR: 0.0635207, NDCG: 0.0256913, F1: 0.0182646, HIT_RATE: 0.1262805, ARHR_ALL_HITS: 0.0698563, NOVELTY: 0.0046243, AVERAGE_POPULARITY: 0.1763813, DIVERSITY_MEAN_INTER_LIST: 0.6180009, DIVERSITY_HERFINDAHL: 0.9617986, COVERAGE_ITEM: 0.0030749, COVERAGE_ITEM_HIT: 0.0013229, ITEMS_IN_GT: 0.8713530, COVERAGE_USER: 0.9825122, COVERAGE_USER_HIT: 0.1240722, USERS_IN_GT: 0.9825122, DIVERSITY_GINI: 0.0008957, SHANNON_ENTROPY: 4.9734586, RATIO_DIVERSITY_HERFINDAHL: 0.9621171, RATIO_DIVERSITY_GINI: 0.0020256, RATIO_SHANNON_ENTROPY: 0.3686862, RATIO_AVERAGE_POPULARITY: 3.4731695, RATIO_NOVELTY: 0.4011369, 

Iteration No: 3 ended. Evaluation done at random point.
T



  0%|          | 0/27968 [00:00<?, ?it/s]

  0%|          | 8/27968 [00:02<2:05:49,  3.70it/s]

  0%|          | 40/27968 [00:02<20:12, 23.04it/s] 

  0%|          | 104/27968 [00:02<07:03, 65.73it/s]

  1%|          | 168/27968 [00:02<03:54, 118.40it/s]

  1%|          | 208/27968 [00:02<03:31, 131.31it/s]

  1%|          | 264/27968 [00:02<02:36, 177.53it/s]

  1%|          | 304/27968 [00:03<02:28, 185.92it/s]

  1%|          | 336/27968 [00:03<02:14, 205.83it/s]

  1%|▏         | 392/27968 [00:03<02:12, 208.40it/s]

  2%|▏         | 424/27968 [00:03<02:05, 220.02it/s]

  2%|▏         | 488/27968 [00:03<01:50, 248.90it/s]

  2%|▏         | 520/27968 [00:03<01:49, 251.76it/s]

  2%|▏         | 584/27968 [00:04<01:44, 263.22it/s]

  2%|▏         | 648/27968 [00:04<01:35, 287.33it/s]

  2%|▏         | 680/27968 [00:04<01:39, 274.46it/s]

  3%|▎         | 744/27968 [00:04<01:31, 296.80it/s]

  3%|▎         | 776/27968 [00:04<01:39, 273.92it/s]

  3%|▎         | 808/27968 [00:04<01:36,

EvaluatorHoldout: Processed 40901 (100.0%) in 20.09 sec. Users per second: 2036
SearchBayesianSkopt: Config 3 is suboptimal. Config: {'alpha': 1.0458622006772091, 'l1_ratio': 0.00581549427616406, 'topK': 975} - results: PRECISION: 0.0220655, PRECISION_RECALL_MIN_DEN: 0.0346081, RECALL: 0.0318892, MAP: 0.0099542, MAP_MIN_DEN: 0.0157922, MRR: 0.0791563, NDCG: 0.0342800, F1: 0.0260830, HIT_RATE: 0.1725386, ARHR_ALL_HITS: 0.0886058, NOVELTY: 0.0041243, AVERAGE_POPULARITY: 0.2586254, DIVERSITY_MEAN_INTER_LIST: 0.7084961, DIVERSITY_HERFINDAHL: 0.9708479, COVERAGE_ITEM: 0.0034682, COVERAGE_ITEM_HIT: 0.0018593, ITEMS_IN_GT: 0.8713530, COVERAGE_USER: 0.9825122, COVERAGE_USER_HIT: 0.1695212, USERS_IN_GT: 0.9825122, DIVERSITY_GINI: 0.0011443, SHANNON_ENTROPY: 5.3362330, RATIO_DIVERSITY_HERFINDAHL: 0.9711694, RATIO_DIVERSITY_GINI: 0.0025877, RATIO_SHANNON_ENTROPY: 0.3955790, RATIO_AVERAGE_POPULARITY: 5.0926594, RATIO_NOVELTY: 0.3577623, 

Iteration No: 4 ended. Evaluation done at random point.
Tim



  0%|          | 0/27968 [00:00<?, ?it/s]

  0%|          | 8/27968 [00:01<1:53:16,  4.11it/s]

  0%|          | 104/27968 [00:02<07:44, 60.01it/s]

  1%|          | 200/27968 [00:02<04:10, 110.89it/s]

  1%|          | 296/27968 [00:02<02:58, 154.77it/s]

  1%|▏         | 392/27968 [00:03<02:24, 190.59it/s]

  2%|▏         | 488/27968 [00:03<02:06, 217.70it/s]

  2%|▏         | 584/27968 [00:03<01:54, 239.58it/s]

  2%|▏         | 680/27968 [00:04<01:46, 255.95it/s]

  3%|▎         | 776/27968 [00:04<01:42, 266.41it/s]

  3%|▎         | 872/27968 [00:04<01:38, 276.37it/s]

  3%|▎         | 968/27968 [00:05<01:35, 282.47it/s]

  4%|▍         | 1064/27968 [00:05<01:33, 288.07it/s]

  4%|▍         | 1160/27968 [00:05<01:32, 289.37it/s]

  4%|▍         | 1256/27968 [00:06<01:31, 290.99it/s]

  5%|▍         | 1352/27968 [00:06<01:30, 292.73it/s]

  5%|▌         | 1448/27968 [00:06<01:29, 295.19it/s]

  6%|▌         | 1544/27968 [00:07<01:28, 297.33it/s]

  6%|▌         | 1640/27968 [00:0

KeyboardInterrupt: 